<a href="https://colab.research.google.com/github/estevaosmith/Portifolio/blob/main/PROJETO_Classificador_de_Fake_News_com_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificador de Fake News


Estamos na era digital, onde o acesso à informação é mais fácil do que nunca, e essa acessibilidade continua crescendo, tornando-se mais rápida e abrangente.

Embora essa evolução traga benefícios, também acarreta problemas e riscos. Neste contexto, um dos maiores desafios com os quais nos deparamos são as Fake News.
<p align=center>
  <img src="https://img.freepik.com/fotos-gratis/conceito-de-noticias-falsas_23-2148836986.jpg?w=740" height="5%">
</p>

Certamente, você já se deparou com uma delas. Em uma era dominada pela informação, a desinformação torna-se um desafio significativo.

Em resumo, Fake News são, como o próprio nome sugere, notícias falsas. Elas podem variar desde uma simples discrepância estatística até a divulgação de eventos completamente inventados.









## Como isso influencia nossa vida?

Como de costume, nós tendemos a acreditar nos grandes veículos de mídia, ou de pessoas do nosso ciclo de amigos que nos encaminham notícias em redes sociais. Dessa forma, criamos o hábito de não checar nossas fontes, e assim, acreditamos e disseminamos notícias falsas, às vezes sem nem saber.

Isso se torna um grande problema quando afeta o resultado de algo importante no bem comum de nossa sociedade, como resultados de eleições, opinião pública sobre determinados projetos, políticas públicas, e até pessoas públicas, sejam elas do meio político, artístico, jornalístico, ou qualquer outro.

## O Projeto

Atualmente, dispomos de ferramentas que auxiliam na identificação e filtragem de notícias falsas. Utilizando técnicas de Machine Learning, como Redes Neurais, torna-se mais viável a tarefa de examinar múltiplas fontes de notícias para discernir entre informações falsas e verdadeiras.

O propósito do projeto é desenvolver um modelo capaz de classificar notícias com a máxima precisão possível, distinguindo entre falsas e factuais. Para alcançar esse objetivo, será empregada a biblioteca TensorFlow para treinar Redes Neurais que desempenhem essa função.

In [2]:
# importando os pacotes necessarios
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

# configurando o noteebook
np.random.seed(42)
sns.set_style()

# filtrar mensagens de warning
import warnings
warnings.filterwarnings('ignore')


### Os Dados
Nosso conjunto de dados é brasileiro, composto por cerca de 12.000 notícias de várias fontes, incluindo UOL, Extra e outras fontes de notícias. A classificação entre fake news ou não foi realizada por diversas agências de verificação de fatos, como a Boatos.org e a Fato ou Fake, entre outras.

**Dicionario de variaveis:**
Title	- Título do artigo

Sub-title - Breve descrição da notícia

News	- Informação sobre o artigo

Category - 	Notícias agrupadas de acordo com suas informações

Author	- Autor da publicação

Date - 	Data de publicação

URL - 	Endereço web do artigo

Class	0-  Notícias falsas

Class 1- Notícias reais

In [3]:
# carregando e verificando os dados
df = pd.read_csv('/content/FakeRecogna (1).csv')

print(df.shape)
df.head()

(11903, 8)


,Titulo,Subtitulo,Noticia,Categoria,Data,Autor,URL,Classe
0,\n\nPapa Francisco foi preso sob acusação de t...,Boato – Ocorreu um apagão no Vaticano. O papa ...,apagão vaticano papar presar acusação tráfico ...,entretenimento,11/01/2021,\nEdgard Matsuki,https://www.boatos.org/religiao/papa-francisco...,0.0
1,Equador prepara cova coletiva para mortos por ...,NaN,o governar equador anunciar preparar cova cole...,saúde,27/03/2020 18h25,27/03/2020 18h25,https://noticias.uol.com.br/internacional/ulti...,1.0
2,Air France voltará a operar voo direto Pequim-...,NaN,o companhia air france operar voar direto pequ...,saúde,07/08/2020 13h42,07/08/2020 13h42,https://www.uol.com.br/nossa/noticias/afp/2020...,1.0
3,Marfrig intensifica venda de carne do Brasil a...,NaN,o marfrig global foods retomar vender carnar b...,saúde,27/04/2020 14h53,27/04/2020 14h53,https://economia.uol.com.br/noticias/reuters/2...,1.0
4,As parciais das eleições de 2014 alternaram ma...,NaN,o assunto voltar o compartilhar rede social ju...,entretenimento,31/07/2021,Gilmar Lopes,https://www.e-farsas.com/as-parciais-das-eleic...,0.0


#Preprocessamento dos dados
Com nosso conjunto de dados em mãos, procederei agora com o pré-processamento para a construção do modelo de Deep Learning. Como lidamos com texto, removerei pontuações e caracteres especiais que não possuem valor algum. Em seguida, converterei as palavras em "tokens" para que nosso modelo possa "compreender" o texto.

In [4]:
df['Noticia'] = df['Noticia'].astype(str)

In [5]:
import pandas as pd
import tensorflow as tf
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('portuguese')

def preprocess_text(text):

    text = text.lower()

    text = re.sub(r'<.*?>', '', text)

    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    tokens = nltk.word_tokenize(text)

    stop_words = set(stopwords.words('portuguese'))
    tokens = [word for word in tokens if word not in stop_words]

    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    return tokens

df['Noticia_preprocessed'] = df['Noticia'].apply(preprocess_text)

print(df['Noticia_preprocessed'].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Error loading portuguese: Package 'portuguese' not found
[nltk_data]     in index


0    [apago, vaticano, papar, presar, acusao, trfic...
1    [governar, equador, anunciar, preparar, cova, ...
2    [companhia, air, franc, operar, voar, direto, ...
3    [marfrig, global, food, retomar, vender, carna...
4    [assunto, voltar, compartilhar, rede, social, ...
Name: Noticia_preprocessed, dtype: object


In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Noticia_preprocessed'])
vocab_size = len(tokenizer.word_index) + 1

In [7]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenized_sequences = tokenizer.texts_to_sequences(df['Noticia_preprocessed'])
max_length = max([len(seq) for seq in tokenized_sequences])

## Modelo de Deep Learning:

Agora com os dados preprocessados, partiremos para o modelo.Vou utilizar as Convolutional Neural Networks (CNNs), elas são uma classe especializada de redes neurais profundas, especialmente projetadas para processar e analisar dados de grade estruturados, como imagens. Elas são amplamente utilizadas em tarefas de visão computacional, como reconhecimento de imagens, detecção de objetos, segmentação de imagens e muito mais.

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=250, input_length=max_length),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout

df = df.dropna(subset=['Classe', 'Noticia_preprocessed'])

train_data, val_data = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Classe'])

train_texts = train_data['Noticia_preprocessed'].values
val_texts = val_data['Noticia_preprocessed'].values

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)

vocab_size = len(tokenizer.word_index) + 1
max_length = max([len(seq) for seq in train_sequences + val_sequences])
embedding_dim = 100

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding='post')

train_labels = train_data['Classe'].values
val_labels = val_data['Classe'].values

def define_model(vocab_size, max_length, embedding_dim):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
        Conv1D(filters=128, kernel_size=5, activation='relu'),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

model = define_model(vocab_size, max_length, embedding_dim)

batch_size = 32
epochs = 10
model.fit(train_padded, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(val_padded, val_labels))

Epoch 1/10
298/298 [==============================] - 47s 145ms/step - loss: 0.3130 - accuracy: 0.8631 - val_loss: 0.1629 - val_accuracy: 0.9366
Epoch 2/10
298/298 [==============================] - 19s 65ms/step - loss: 0.0799 - accuracy: 0.9758 - val_loss: 0.1732 - val_accuracy: 0.9425
Epoch 3/10
298/298 [==============================] - 17s 59ms/step - loss: 0.0263 - accuracy: 0.9924 - val_loss: 0.2510 - val_accuracy: 0.9366
Epoch 4/10
298/298 [==============================] - 12s 40ms/step - loss: 0.0218 - accuracy: 0.9942 - val_loss: 0.2652 - val_accuracy: 0.9353
Epoch 5/10
298/298 [==============================] - 13s 42ms/step - loss: 0.0139 - accuracy: 0.9960 - val_loss: 0.3506 - val_accuracy: 0.9349
Epoch 6/10
298/298 [==============================] - 7s 24ms/step - loss: 0.0094 - accuracy: 0.9976 - val_loss: 0.3873 - val_accuracy: 0.9362
Epoch 7/10
298/298 [==============================] - 8s 27ms/step - loss: 0.0046 - accuracy: 0.9982 - val_loss: 0.4800 - val_accuracy: 

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


test_texts = val_data['Noticia_preprocessed'].values
test_labels =val_data['Classe'].values

test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

test_predictions = model.predict(test_padded)
test_predictions = (test_predictions > 0.5).astype(int)

test_accuracy = accuracy_score(test_labels, test_predictions)
test_precision = precision_score(test_labels, test_predictions)
test_recall = recall_score(test_labels, test_predictions)
test_f1 = f1_score(test_labels, test_predictions)

print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1-Score: {test_f1:.4f}")

75/75 [==============================] - 0s 5ms/step
Test Accuracy: 0.9324
Test Precision: 0.9305
Test Recall: 0.9345
Test F1-Score: 0.9325


Como podem observar, **nosso modelo obteve uma acurácia de 93%!** Isso pode indicar um modelo satisfatório, mas também pode estar sofrendo de overfitting. No entanto, devido à extensão considerável do conjunto de dados e à diversidade das notícias e autores, acredito ser pouco provável que o resultado esteja sendo afetado pelo sobreajuste.

## Conclusão
Concluindo, o desenvolvimento e a implementação de um modelo de classificação de Fake News utilizando Deep Learning, especificamente a biblioteca TensorFlow, demonstrou ser uma abordagem promissora para lidar com o desafio crescente de identificar informações falsas em meio a um fluxo de notícias. A acurácia de 93% alcançada pelo modelo de Convolutional Neural Networks (CNNs) é encorajadora, sugerindo que o modelo está eficazmente discernindo entre notícias verdadeiras e falsas.

A etapa inicial de pré-processamento, que envolve a remoção de pontuações e caracteres especiais sem valor, seguida pela conversão das palavras em "tokens", se mostrou fundamental para preparar os dados de entrada para o modelo de Deep Learning.


Embora a possibilidade de overfitting seja uma preocupação válida, a diversidade do conjunto de dados utilizado e a considerável quantidade de exemplos sugerem que o impacto do sobreajuste é mitigado. No entanto, é prudente continuar monitorando o desempenho do modelo e realizar ajustes conforme necessário.

No contexto atual, onde a disseminação de desinformação é uma preocupação global, a aplicação de técnicas avançadas de Machine Learning, como Redes Neurais, pode desempenhar um papel crucial na proteção da integridade da informação. Este estudo representa um passo significativo em direção à construção de sistemas mais robustos e eficazes para identificar e combater Fake News.